# Initialize the notebook and download some molecules

In [2]:
from preamble import *

2025-02-02 08:39:27,435 - WARNING - File .dacerc not found. You are requesting data in public mode. To change this behaviour, create a .dacerc file in your home directory and fill it with your API key. More infos on https://dace.unige.ch


  0%|          | 0/2 [00:00<?, ?it/s]

# Download the proper chemicals

In [4]:
# download_molecule(
#     molecule_name='H2O',
#     temperature_range=[300, 1500],
#     pressure_range=[-8, 2]
# )
# download_molecule(
#     molecule_name='CO2',
#     temperature_range=[300, 1500],
#     pressure_range=[-8, 2]
# )
# download_molecule(
#     molecule_name='CH4',
#     temperature_range=[300, 1500],
#     pressure_range=[-8, 2]
# )

In [45]:
Opacity.get_available_species()

name,species,charge,line_list,version,size_gb,path,index
str3,str10,int64,str9,int64,float64,str59,int64
CH4,12C-1H4,--,YT34to10,1,1.622,/Users/wiwa8630/.shone/12C-1H4_YT34to10_400_1200_-7_2_1.nc,0
CO2,12C-16O2,--,UCL-4000,1,2.704,/Users/wiwa8630/.shone/12C-16O2_UCL-4000_400_1200_-7_2_1.nc,1
H2O,1H2-16O,--,POKAZATEL,2,5.678,/Users/wiwa8630/.shone/1H2-16O_POKAZATEL_400_1200_-7_2_2.nc,2
HCN,1H-12C-14N,--,Harris,2,2.434,/Users/wiwa8630/.shone/1H-12C-14N_Harris_400_1200_-7_2_2.nc,3


# Define the Model Components
 (1) HST Ramp effect
 
 (2) Stellar Rotation and Planetary Transit (fleck)
 
 (3) HST Breathing Effect (if it seems necessary)

## With these model components, the total combined model should look something like this:

$$ F(\rm t, \lambda) = M_{\rm systematics}(\Theta_{\rm systematics}) \times M_{\rm star+transit}(\Theta_{\rm star+transit}) $$

# Define the transmission model

In [9]:
# https://shone.readthedocs.io/en/latest/shone/examples/transmission.html#general-transmission-spectra
def transmission_spectrum_HengKitz17(log_atm_pressure = -1,
                                     atm_temp = 600 * u.K,
                                     log_kappa_cloud = -2,  # cloud opacity [cm2 / g]
                                     mmw=2.5,
                                     R_0=3.5 * u.R_earth,
                                     plot=False):
    """
    Compute a transmission spectrum for an atmosphere
    using the isothermal/isobaric approximation
    from Heng & Kitzmann (2017).
    """

    P_0 = 10**log_atm_pressure
    temperature = jnp.array([atm_temp.value])  # [K]
    pressure = jnp.array([P_0]) # [bar]
    chem = FastchemWrapper(temperature, pressure)
    # chem = build_fastchem_interpolator(temperature, pressure)
    vmr = chem.vmr()
    weights = chem.get_weights()

    weighted_opacities = []
    for i, spec in enumerate(species):
        op = binned_opacities[i](atm_temp.value, P_0)[0]  # cm2 / g
        col_idx = chem.get_column_index(species_name=spec)[0]
        species_weight = weights[col_idx] / mmw
    
        abund_weighted_opacity = op * species_weight * vmr[:, col_idx]
        weighted_opacities.append(abund_weighted_opacity)
    
    g = ( (c.G * M_p) / (R_0)**2 ).decompose() # surface gravity
    
    # compute the planetary radius as a function of wavelength:
    Rp = heng_kitzmann_2017.transmission_radius_isothermal_isobaric(
        weighted_opacities[0] + weighted_opacities[1] + (10**log_kappa_cloud),
        R_0.cgs.value, P_0, atm_temp, mmw, g.cgs.value
    )

    # convert to transit depth:
    transit_depth_ppm = 1e6 * (Rp / Rs.cgs.value) ** 2

    if plot:
        plt.figure()
        plt.errorbar(wavelengths, transit_depth_ppm, yerr=40e-6)
        plt.ylabel('Transit Depth (ppm)')
        plt.show()
        plt.clf()
        
    return (Rp / Rs.cgs.value), transit_depth_ppm

# Example Usage:
# Rp, transit_depth_ppm = transmission_spectrum_HengKitz17()

# Define the log probability

In [28]:
def lnprob(parameters, plot=False, **kwargs):
    """
    Log-probability function for MCMC.

    Parameters:
    - parameters: np.ndarray, current parameter values.
    - kwargs: dict, additional arguments required for model computation.

    Returns:
    - ln_like: float, log-likelihood of the current parameter set.
    """
    ln_like = 0.0

    # Map parameters to names for easier reference
    params = {name: value for name, value in zip(params_config.keys(), parameters)}
    
    'Check if all priors are satisfied'
    if all(bounds[0] <= params[key] <= bounds[1] for key, bounds in priors.items()):

        'Set binning preferences and download model spectra'
        kwargs = dict(
        bin_specification = 'edges',
        bins=bin_edges,
        min=bin_edges.min(),
        max=bin_edges.max(), log=False)
        Cool, Phot = [
            bin_spectrum(
                get_spectrum(T_eff=Temp, log_g=log_g, Z=0.12, cache=True), **kwargs
            )
            for Temp in [params["T_spot"], params["T_phot"]] ]

        'Update Planet Parameters'
        planet_parameters['a'] = params['a_rstar']
        planet_parameters['ecc'] = params['ecc']
        planet_parameters['inclination'] = np.radians(params['planet_i'])
        planet_parameters['u1'] = np.array([params[f"u1_w{i}"] for i in range(len(wavelengths))])
        planet_parameters['u2'] = np.array([params[f"u2_w{i}"] for i in range(len(wavelengths))])
        # planet_parameters['rp'] = ( (params['R_0']*u.R_earth)/(Rs*u.R_sun) ).decompose() # transmission_spectrum(vmr=10 ** log_vmr, mmw=mmw)[0]
        planet_parameters['rp'] = transmission_spectrum_HengKitz17(R_0 = params['R_0'] * u.R_earth,
                                                        log_atm_pressure = params['log_P_atm'],
                                                        atm_temp = params['T_atm'] * u.K,
                                                        log_kappa_cloud = params['log_kappa_cloud'],
                                                        mmw = params['mmw'] )[0]
        if visit == 'S22':
            planet_parameters['t0'] = params['T_0']
            
        'Update stellar parameters'
        active_star.inclination = np.radians(params['stellar_i'])
        active_star.T_eff = Phot.meta['PHXTEFF']
        active_star.phot = Phot.flux.value
        active_star.wavelength = Phot.wavelength.to_value(u.m)
        active_star.temperature = jnp.array([Cool.meta['PHXTEFF']]*n_spots)
        active_star.spectrum = jnp.array([Cool.flux.value]*n_spots)
        active_star.lon = jnp.array([params[f"spot{i}_lon"] for i in range(1, n_spots + 1)])
        active_star.lat = jnp.array([params[f"spot{i}_lat"] for i in range(1, n_spots + 1)])
        active_star.rad = jnp.array([params[f"spot{i}_rad"] for i in range(1, n_spots + 1)])


        for data_flux, relative_err, img_date, time_from_T0, direction in zip(data_fluxes, relative_errs, img_dates, times_from_T0, ['Forward','Backward']):
            active_star.time = time_from_T0
            lc, contam, X, Y, spectrum_at_transit = active_star.transit_model(**planet_parameters)
            # sh = np.array(sh_data[f"{visit}_{direction}"]["lightcurve"]['Shift'])
            
            systematics = ramp_model(params['r1'],params['r2'],img_date)
    

            if plot:

                active_star.plot_star(t0 = 0,
                                      rp = ( params['R_0'] * u.R_earth / Rs ).decompose(),#Exoplanet radius in units of stellar radii
                                      a = params['a_rstar'],#Planetary semi-major axis in units of stellar radii
                                      inclination=1.56, #Planetary orbital inclination [radians]
                                      ecc = params['ecc'])

                plt.figure()
                plt.title('Transmission Spectrum')
                plt.scatter(wavelengths, planet_parameters['rp'])
                plt.show()
                plt.clf()

                # print(systematics)
                # print('systematics shape:', systematics.shape)
                plt.plot(img_date,systematics)
                plt.title('Systematics')
                plt.show()
                plt.clf()
                      
                plt.figure()
                plt.title('Contamination Spectrum')
                plt.scatter(wavelengths, contam)
                plt.show()
                plt.clf()
                # print('These Params:')
                # print(params)
                
                fig, axes = plt.subplots(1, 3, figsize=(14, 3), gridspec_kw={'width_ratios': [1, 1, 1]})
                axes[0].set_title(f"{visit} {direction} with model")
                axes[0].set_xlabel("Time from midtransit")
                axes[0].set_ylabel("Relative Flux")
                axes[1].set_title("Systematic-corrected")
                axes[1].set_xlabel("Time from midtransit")
                axes[2].set_title("Residuals")
                axes[2].set_xlabel("Time from midtransit")
                axes[2].set_ylabel(r"$\sigma$")

                chisq_per_wavelength = []
                for i, lc_i in enumerate(lc.T):
                    lc_i /= np.nanmean(lc_i)
                    model_i = systematics * lc_i
                    model_i /= np.nanmean(model_i)

                    flux_i = data_flux[i, :]
                    err_i = relative_err[i, :]
                    normalized_flux = flux_i/np.nanmean(flux_i)
                    chisq = np.nansum(((normalized_flux - model_i) ** 2) / (err_i ** 2))
                    chisq_per_wavelength.append(chisq)
                    
                    # First plot (data + model)
                    axes[0].errorbar(
                        time_from_T0,
                        normalized_flux + (0.0015*i),
                        yerr=err_i,
                        color=plt.cm.turbo((wavelengths[i] - wavelengths.min()) / np.ptp(wavelengths)))
                    axes[0].plot(
                        time_from_T0,
                        model_i + (0.0015 * i),
                        color='k',zorder=100)
                    # Second plot (data & model)/systematics
                    axes[1].errorbar(
                        time_from_T0,
                        (normalized_flux/systematics) + (0.0015*i),
                        yerr=err_i,
                        color=plt.cm.turbo((wavelengths[i] - wavelengths.min()) / np.ptp(wavelengths)))
                    axes[1].plot(
                        time_from_T0,
                        lc_i + (0.0015 * i),
                        color='k',zorder=100)
                    # third plot (data & model)/systematics
                    axes[2].scatter(
                        time_from_T0,
                        (normalized_flux-model_i)/err_i + (0.0015 * i), alpha=0.3,
                        color=plt.cm.turbo((wavelengths[i] - wavelengths.min()) / np.ptp(wavelengths)))
                
                plt.savefig(f'../figs/{visit}_{direction}_lightcurvemodels.png')
                plt.show()
                
                reduced_chisq = np.nansum(chisq_per_wavelength)/( (np.sum(~np.isnan(wavelengths)) * np.sum(~np.isnan(time_from_T0))) - ndim)
                print(f'This Models Reduced Chisq = {reduced_chisq} for {ndim} parameters and {(np.sum(~np.isnan(wavelengths)) * np.sum(~np.isnan(time_from_T0)))} data points')
    
            for i, lc_i in enumerate(lc.T):
                lc_i /= np.nanmean(lc_i)
                model_i = systematics * lc_i
                model_i /= np.nanmean(model_i)

                flux_i = data_flux[i, :]
                err_i = relative_err[i, :]
                normalized_flux = flux_i/np.nanmean(flux_i)
                chisq = np.nansum(((normalized_flux - model_i) ** 2) / (err_i ** 2))
                
                err_weight = np.nansum(1.0 / np.sqrt(2.0 * np.pi * (err_i)))
    
                ln_like += (err_weight - 0.5 * chisq)

        return ln_like

    else:
        return -np.inf

# Initialize MCMC parameters and other details

In [35]:
Rs = 0.8 * u.R_sun
log_g = calculate_logg(Rs.value)
T_eq = 600
M_p = 10.2 * u.R_earth 
n_spots = 3
orbits_to_exclude = np.array([0])
err_inflation = 2.0

F21_bin_edges = np.array([1.13, 1.22, 1.285, 1.46, 1.565, 1.64])*u.micron
S22_bin_edges = np.array([0.81, 0.86, 0.92, 0.98, 1.04, 1.10, 1.13])*u.micron

nwalkers = 250
nsteps = 1000
burnin = int(0.5*nsteps)

# Configuration for parameter initialization
params_config = {
    # Systematics Model Parameters
    "r1": (11, 13),
    "r2": (-7, -6),
    "r3": (-0.1, 0),
    # "b1": (-1, 1),
    # "b2": (-1, 1),
    # "b3": (-1, 1),
    # "b4": (-1, 1),
    # "l1": (-1, 1),
    # "l2": (-1, 1),
    # "l3": (-1, 1),
    # "l4": (-1, 1),
    # "slope_Forward": (-1,1),
    # "slope_Backward": (-1,1),
    # Transit Parameters -> u1 and u2 are defined later
    "R_0": (3, 4),
    "a_rstar": (19, 20),
    "ecc": (0.05, 0.2),
    "planet_i": (89,90),
    # Transmission Parameters
    "log_P_atm": (-3,-1),
    "T_atm": (500, 700),
    "mmw": (2,20),
    "log_kappa_cloud": (-3,-1),
    # Spot Parameters
    "stellar_i": (85, 90),
    "T_spot": (2900, 3200),
    "T_phot": (3800, 4200),
    "spot1_lon": (0.8, 0.82),
    "spot1_lat": (1.96, 1.99),
    "spot1_rad": (0.2, 0.3),
    "spot2_lon": (-0.8, -0.4),
    "spot2_lat": (0.9, 1.1),
    "spot2_rad": (0.25, 0.3),
    "spot3_lon": (-1.49, -1.4),
    "spot3_lat": (0.3, 0.5),
    "spot3_rad": (0.4, 0.45)}

# Define uniform priors
priors = {
    "r1": (1, 30),
    "r2": (-10, 0),
    "r3": (-1, 1),
    # "b1": (-10, 10),
    # "b2": (-10, 10),
    # "b3": (-10, 10),
    # "b4": (-10, 10),
    # "l1": (-10, 10),
    # "l2": (-10, 10),
    # "l3": (-10, 10),
    # "l4": (-10, 10),
    # Transit
    "R_0": (2.0, 6.0),
    "a_rstar": (17.0, 21.0),
    "ecc": (0.0, 0.4),
    # Transmission
    "log_P_atm": (-7, 2),
    "T_atm": (300, 1500),
    "mmw": (2,60),
    "log_kappa_cloud": (-7,0),
    # Spot
    "T_spot": (2500, 3600),
    "T_phot": (3600, 4500)}

for i in range(1, 4):
    priors[f"spot{i}_lon"] = (-np.pi, np.pi)
    priors[f"spot{i}_lat"] = (0.0, np.pi)
    priors[f"spot{i}_rad"] = (0.0, 1.0)

# Run the MCMC

In [33]:
for visit in ['F21']:

    print('Running ',visit)
    print('')
    
    if visit == 'F21':
        bin_edges = F21_bin_edges
    if visit == 'S22':
        bin_edges = S22_bin_edges
        params_config["T_0"] = (-0.005, 0.005)
        priors["T_0"] = (-0.05, 0.05)
        
    'Extract exposure times and mid-transit times from the dictionary'
    predicted_T0 = visits[f'{visit}']['T0 (BJD_TDB)'].value
    exposureTime = visits[f'{visit}']['exp (s)'].value

    'Read in the trimmed rainbows from process_pacman_spectra.ipynb'
    data_fluxes = []
    relative_errs = []
    img_dates = []
    times_from_T0 = []
    wave_arrays = []
    for direction in ['Forward','Reverse']:
        rainbow = read_rainbow(f"../data/{visit}_{direction}_trimmed_pacman_spec.rainbow.npy")
        rainbow.uncertainty = np.sqrt(rainbow.flux.value)
        binned_rainbow = rainbow.bin(wavelength_edges=bin_edges)
        wave_arrays.append(binned_rainbow.wavelength.value)
        data_fluxes.append(binned_rainbow.flux.value)
        relative_errs.append(err_inflation * (binned_rainbow.uncertainty/binned_rainbow.flux.value))
        img_dates.append(binned_rainbow.time.value)
        times_from_T0.append(binned_rainbow.time.value - predicted_T0)
    wavelengths = wave_arrays[0] #The Forward-scan wavelengths
    
    'Label the orbits and scan directions for RECTE'
    orbit = np.zeros_like(img_dates[0])
    for j in range(len(img_dates[0])):
        if j >= 1:
            if (img_dates[0][j] - img_dates[0][j - 1]) > 0.025:
                orbit[j] = (orbit[j - 1] + 1)
            else:
                orbit[j] = orbit[j - 1]

    'Trim the first point from each orbit'
    ramp_ref_times = []
    for o in np.unique(orbit):
        first_index = np.where(orbit == o)[0][0]
        ramp_ref_times.append(img_dates[0][first_index])
        for j in range(2):
            data_fluxes[j][:, first_index] = np.nan  # Set the first point of each subsequent orbit to np.nan
            relative_errs[j][:, first_index] = np.nan  # Set the first point of each subsequent orbit to np.nan
            img_dates[j][first_index] = np.nan  # Set the first point of each subsequent orbit to np.nan
            times_from_T0[j][first_index] = np.nan  # Set the first point of each subsequent orbit to np.nan

    'Set data to nan if it was in the pre-defined list of orbits to exclude'
    for orbit_to_exclude in orbits_to_exclude:
        for j in range(2):
            data_fluxes[j][:, orbit == orbit_to_exclude] = np.nan
            relative_errs[j][:, orbit == orbit_to_exclude] = np.nan
            img_dates[j][orbit == orbit_to_exclude] = np.nan
            times_from_T0[j][orbit == orbit_to_exclude] = np.nan
        orbit[orbit==orbit_to_exclude] = np.nan
            
    'Remove nan data'
    for j in range(2):
        # Mask to identify non-NaN values
        mask = ~np.isnan(img_dates[j])
        # print(mask)
        # Redefine each array by excluding NaN values
        data_fluxes[j] = data_fluxes[j][:, mask]
        relative_errs[j] = relative_errs[j][:, mask]
        img_dates[j] = img_dates[j][mask]
        times_from_T0[j] = times_from_T0[j][mask]
    orbits = orbit[mask]
    
    'Generate an opacity grid from a range of wavelengths, pressures, and temperatures'
    pressures = np.geomspace(1e-7, 100, 1000)
    temperatures = T_eq * (pressures / 0.1) ** 0.08
    species = ['H2O', 'CH4', 'CO2', 'CO',
               'NH3', 'OH', 'C2H2']
    binned_opacities = []
    for spec in species:
        op = Opacity.load_species_from_name(spec)
        binned_opacities.append(
            op.get_binned_interpolator(wavelengths, temperatures, pressures) )
        
    'Set binning preferences and download initial spectra'
    kwargs = dict(
        bin_specification = 'edges',
        bins=bin_edges,
        min=bin_edges.min(),
        max=bin_edges.max(),
        log=False)
    Cool, Phot = [
        bin_spectrum(
            get_spectrum(T_eff=Temp, log_g=log_g, Z=0.12, cache=True), **kwargs)
        for Temp in [3100, 4000] ]

    'Initialize planet parameters and the active_star'
    planet_parameters = dict(
        inclination = 1.56,
        a = 18.5,
        rp = 0.048,
        period = 8.463,
        t0 = 0,
        ecc = 0.1,
        u1 = 0.15*np.ones_like(wavelengths),
        u2 = 0.17*np.ones_like(wavelengths) )
    active_star = ActiveStar(
        times=times_from_T0[0], # use data time array
        inclination=np.pi/2,  # stellar inc [rad]
        T_eff=Phot.meta['PHXTEFF'],
        wavelength=Phot.wavelength.to_value(u.m), #use data wavelength array
        phot=Phot.flux.value, #This is the model spectrum for the photosphere temp
        P_rot=4.863 )

    # Generate and add spots dynamically
    for i in range(1, n_spots + 1):
        # Generate random values within the parameter space for each spot
        lon = np.random.uniform(*params_config[f"spot{i}_lon"])
        lat = np.random.uniform(*params_config[f"spot{i}_lat"])
        rad = np.random.uniform(*params_config[f"spot{i}_rad"])
        
        # Define the spot dictionary
        spot = dict(
            lon=lon,  # Longitude [rad]
            lat=lat,  # Latitude [rad]
            rad=rad,  # Radius [R_star]
            spectrum=Cool.flux.value,  # Use provided spectrum
            temperature=Cool.meta['PHXTEFF'],  # Use provided temperature
        )
        # Add the spot to the active star
        active_star.add_spot(**spot)

    'Add wavelength-dependent u1 and u2 parameters'
    for i in range(len(bin_edges)-1):
        params_config[f"u1_w{i}"] = (0.12, 0.18)
        params_config[f"u2_w{i}"] = (0.12, 0.18)
        # Add the same uniform prior for all `u1_wi` and `u2_wi`
        priors[f"u1_w{i}"] = (0.05, 0.5)
        priors[f"u2_w{i}"] = (0.05, 0.5)

    variable_names = list(params_config.keys())
    ndim = len(params_config)
    p0 = initialize_walkers(nwalkers, params_config)

    while True:
        # Select a random set of parameters
        example_params = p0[np.random.randint(0, nwalkers - 1), :]
        
        # Call lnprob with the selected parameters and plot=True
        lnprob(example_params, plot=True)
        
        # Ask the user if they want to repeat
        user_input = input("Do you want to choose another random set of parameters to model? (yes/no): ").strip().lower()
        if user_input != "yes":
            print("Exiting...")
            break
    
    
    'Set up MCMC'
    label=f'{visit}_{nsteps}steps_{int(len(bin_edges)-1)}bins_{n_spots}spots_model'
    samples_fname = f"../data/samples/{label}.h5"
    backend = emcee.backends.HDFBackend(samples_fname)
    backend.reset(nwalkers, ndim)        
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob,backend=backend)

    'Run the MCMC'
    print('Running MCMC sampler...')
    result = sampler.run_mcmc(p0, nsteps, store=True, progress=True)
    samples = sampler.chain[:, burnin:, :].reshape((-1, ndim)).T

    'Check for convergence (all should be > 100)'
    for i in range(len(samples)):
        tau_f = emcee.autocorr.integrated_time(samples[i])
        print('(Nsteps-burnin)*nwalkers/tau=',int((nsteps-burnin)*nwalkers/tau_f))

    'Return the Most likely set of parameters'
    flat_log_prob = sampler.get_log_prob(discard=burnin, flat=True)
    best_index = np.argmax(flat_log_prob)  # Index of the highest log-probability
    highest_prob_params = samples[:, best_index]
    print("Best-fit parameter set:", highest_prob_params)
    lnprob(highest_prob_params, plot=True)
    
    'Extract and display best-fit parameters and uncertainties'
    median_params = []
    err_upper = []
    err_lower = []
    for i, param_samples in enumerate(samples):
        sig_1_results = np.percentile(param_samples, [15.9, 50.0, 84.1])
        median_params.append(sig_1_results[1])
        err_upper.append(sig_1_results[2] - sig_1_results[1])
        err_lower.append(sig_1_results[1] - sig_1_results[0])
    params_dict = {name: median_params[i] for i, name in enumerate(params_config.keys())}
    for i, label in enumerate(variable_names):
        print(f"{label}: {median_params[i]:.6f} +{err_upper[i]:.6f} -{err_lower[i]:.6f}")
        print("--------------")

    'Make corner plot'
    rng = 0.9995
    fig = corner.corner( 
        samples.T,show_titles=True, labels=variable_names,
        range=[rng]*ndim,smooth=1,quantiles=(0.16, 0.5, 0.84),
        fill_contours=True, plot_datapoints=False,title_kwargs={"fontsize": 15},title_fmt='.3f',
        hist_kwargs={"linewidth": 2.5},levels=[(1-np.exp(-0.5)),(1-np.exp(-2)),(1-np.exp(-4.5))])
    plt.suptitle(f"{visit} MCMC Results")
    plt.savefig(f'../figs/{visit}_{nsteps}nsteps_corner.png')
    plt.show()

Running  F21



  0%|          | 0/162 [00:00<?, ?it/s]

  0%|          | 0/162 [00:00<?, ?it/s]

Running MCMC sampler...
emcee: Exception while calling your likelihood function:
  params: [ 1.13944829e+01 -6.15675914e+00 -8.47957351e-02  3.30831890e+00
  1.97080082e+01  5.59138957e-02 -2.87669789e+00  5.63972606e+02
  1.26451727e+01 -1.51289840e+00  3.13498229e+03  3.96002054e+03
  8.03115526e-01  1.96969748e+00  2.51266162e-01 -5.62865087e-01
  9.89465413e-01  2.64237884e-01 -1.44558395e+00  4.47445390e-01
  4.18649088e-01  1.37000661e-01  1.36081407e-01  1.72018940e-01
  1.49777507e-01  1.79674163e-01  1.24634041e-01  1.67298845e-01
  1.30008696e-01  1.66138832e-01  1.73608449e-01]
  args: []
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/opt/anaconda3/envs/fleckopy/lib/python3.10/site-packages/emcee/ensemble.py", line 640, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/var/folders/vq/zxq380ws52q37mjjtbt9c7rh0000gp/T/ipykernel_38305/2923182510.py", line 35, in lnprob
    planet_parameters['inclination'] = np.radians(params['planet_i'])
KeyError: 'planet_i'


KeyError: 'planet_i'